In [1]:
import numpy as np

from mechcell import tracking_object

### Load example image

In [2]:
stack = np.load("stack_tracking_example.npy")

### Extract condensates

In [3]:
track_cost_cutoff = 130**2
splitting_cost_cutoff = 10**2
threshold = 16500
y_res = 0.0339  # from image metadata
x_res = 0.0339  # from image metadata
nr_images = stack.shape[0]

track_df, viewer, labels_per_frame = tracking_object(
    stack, track_cost_cutoff, splitting_cost_cutoff, threshold, y_res, x_res, nr_images
)

/opt/anaconda3/envs/mechanoconf_pinned/lib/python3.11/site-packages/laptrack/_tracking.py:788: FutureWarning: The parameter only_coordinate_cols will be False by default in the major release.
  warnings.warn(


### Check track ids from visualization

In [4]:
import pandas as pd

# Condensate 10:
track_id_4 = track_df.loc[(track_df["track_id"] == 4) & track_df["frame"].between(0, 10)].copy()
track_id_22 = track_df.loc[(track_df["track_id"] == 22) & track_df["frame"].between(11, 23)].copy()
track_id_43 = track_df.loc[(track_df["track_id"] == 43) & track_df["frame"].isin([24, 25])].copy()
track_id_72 = track_df.loc[(track_df["track_id"] == 72) & (track_df["frame"] == 26)].copy()

track_df_cond_10 = pd.concat([track_id_4, track_id_22, track_id_43, track_id_72])
track_df_cond_10 = track_df_cond_10.sort_values(by="frame")
track_df_cond_10.loc[:, "Condensate ID"] = 10

# Condensate 11:
track_df_cond_11 = track_df.loc[(track_df["track_id"] == 5) & track_df["frame"].between(1, 27)].copy()
track_df_cond_11.loc[:, "Condensate ID"] = 11

# Condensate 12:
track_df_cond_12 = track_df.loc[(track_df["track_id"] == 7) & track_df["frame"].between(1, 17)].copy()
track_df_cond_12.loc[:, "Condensate ID"] = 12

# Condensate 13:
track_id_2 = track_df.loc[(track_df["track_id"] == 2) & track_df["frame"].isin([1, 2])].copy()
track_id_10 = track_df.loc[(track_df["track_id"] == 10) & track_df["frame"].isin([3, 5])].copy()
track_id_3 = track_df.loc[(track_df["track_id"] == 3) & (track_df["frame"] == 4)].copy()
track_id_17 = track_df.loc[(track_df["track_id"] == 17) & track_df["frame"].isin([10, 11, 12])].copy()
track_id_19 = track_df.loc[(track_df["track_id"] == 19) & (track_df["frame"] == 13)].copy()

track_df_cond_13 = pd.concat([track_id_2, track_id_10, track_id_3, track_id_17, track_id_19])
track_df_cond_13 = track_df_cond_13.sort_values(by="frame")
track_df_cond_13.loc[:, "Condensate ID"] = 13

# Combined to nucleus
track_df = pd.concat([track_df_cond_10, track_df_cond_11, track_df_cond_12, track_df_cond_13])
track_df.loc[:, "Nucleus ID"] = 3

track_df

,centroid_y,centroid_x,frame,track_id,index,Condensate ID,Nucleus ID
4,813.773842,277.824251,0,4,5,10,3
12,819.442029,241.509662,1,4,8,10,3
19,820.118220,218.944367,2,4,7,10,3
30,821.402926,236.031915,3,4,11,10,3
44,820.831633,247.391156,4,4,14,10,3
...,...,...,...,...,...,...,...
52,672.562937,290.199301,5,10,8,13,3
141,684.740385,284.150641,10,17,16,13,3
162,678.512586,291.405034,11,17,13,13,3
191,682.963636,274.994805,12,17,21,13,3


### Visualize tracks

In [5]:
viewer.add_tracks(
    track_df[["Condensate ID", "frame", "centroid_x", "centroid_y"]],
    # graph=graph,
    scale=(y_res, x_res),
)

new_labels = np.zeros_like(labels_per_frame)

for i, row in track_df.iterrows():
    frame = int(row["frame"])
    inds = labels_per_frame[frame] == row["index"]
    new_labels[frame][inds] = int(row["Condensate ID"]) + 1

viewer.add_labels(new_labels, scale=(y_res, x_res))

<Labels layer 'new_labels [1]' at 0x1e474f010>

### Extract nuclei

In [6]:
track_cost_cutoff = 130**2
splitting_cost_cutoff = 10**2
threshold_nuc = 5500

track_df_nuc, viewer, labels_per_frame = tracking_object(
    stack, track_cost_cutoff, splitting_cost_cutoff, threshold_nuc, y_res, x_res, nr_images
)

track_df_nuc["Nucleus ID"] = 3

/opt/anaconda3/envs/mechanoconf_pinned/lib/python3.11/site-packages/laptrack/_tracking.py:788: FutureWarning: The parameter only_coordinate_cols will be False by default in the major release.
  warnings.warn(


### Beautify dataframe

In [7]:
merged = track_df_nuc.merge(track_df, on=["Nucleus ID", "frame"])

# Rename for condensates and nuclei
merged = merged.rename(
    columns={
        "centroid_y_x": "centroid-y_nuc",
        "centroid_x_x": "centroid-x_nuc",
        "centroid_y_y": "centroid-y_cond",
        "centroid_x_y": "centroid-x_cond",
        "track_id_y": "track_id",
    }
)

# Adjust for nuclei movement for condensates
merged["y_corr"] = merged["centroid-y_cond"] - merged["centroid-y_nuc"]
merged["x_corr"] = merged["centroid-x_cond"] - merged["centroid-x_nuc"]

# Scale the coordinates
merged["y_corr"] = merged["y_corr"] * y_res
merged["x_corr"] = merged["x_corr"] * x_res

# Add condition and video ID
merged["Condition"] = "Confined"
merged["Video"] = "3626"

merged = merged[["Video", "Condition", "Nucleus ID", "Condensate ID", "frame", "track_id", "x_corr", "y_corr"]]

merged

,Video,Condition,Nucleus ID,Condensate ID,frame,track_id,x_corr,y_corr
0,3626,Confined,3,10,0,4,-0.574969,6.495878
1,3626,Confined,3,10,1,4,-1.165382,6.374272
2,3626,Confined,3,11,1,5,-0.229811,-6.908069
3,3626,Confined,3,12,1,7,2.748592,5.129694
4,3626,Confined,3,13,1,2,-0.692096,1.563245
...,...,...,...,...,...,...,...,...
77,3626,Confined,3,10,25,43,-0.554698,5.443862
78,3626,Confined,3,11,25,5,-1.330697,-5.487754
79,3626,Confined,3,10,26,72,-1.010359,5.189859
80,3626,Confined,3,11,26,5,-1.451170,-5.525722


### Save dataframe

In [8]:
merged.to_csv("data_tracking_condensates.csv", index=False, header=True)